<a href="https://colab.research.google.com/github/ykato27/Text-to-Speech/blob/main/ch11_Advanced_demos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced TTS demos


このページ（ノートブック形式）では、第11章で少し触れた「非自己回帰型ニューラルボコーダ」を用いた、発展的な音声合成のデモを示します。
書籍ではJSUTコーパスのみを扱いましたが、ここではJVSコーパスを用いた多話者音声合成など、他のコーパスを利用した音声合成のデモも紹介します。
このページのデモは、書籍では解説していないことに注意してください。

非自己回帰型ニューラルボコーダの実装には、[kan-bayashi/ParallelWaveGAN](https://github.com/kan-bayashi/ParallelWaveGAN) を利用します。
多話者音声合成の実装は、書籍では実装の解説はしていませんが、第9章、第10章の内容に、軽微な修正を加えることで実現可能です。
興味のある読者は、extra_recipes のソースコードを参照してください。

## 準備

### ttslearn のインストール

In [ ]:
%%capture
try:
    import ttslearn
except ImportError:
    !pip install ttslearn

In [ ]:
import ttslearn
ttslearn.__version__

### パッケージのインポート

In [ ]:
%pylab inline
import IPython
from IPython.display import Audio
import librosa
import librosa.display
from tqdm.notebook import tqdm
import torch
import random

## JSUT

### Tacotron + Parallel WaveGAN (16kHz）

In [ ]:
from ttslearn.contrib import Tacotron2PWGTTS

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Device:", device)

pwg_engine = Tacotron2PWGTTS(device=device)

%time wav, sr = pwg_engine.tts("あらゆる現実を、すべて自分のほうへねじ曲げたのだ。")
IPython.display.display(Audio(wav, rate=sr))

### Tacotron + Parallel WaveGAN (24kHz）

In [ ]:
from ttslearn.pretrained import create_tts_engine

pwg_engine = create_tts_engine("tacotron2_pwg_jsut24k", device=device)

%time wav, sr = pwg_engine.tts("あらゆる現実を、すべて自分のほうへねじ曲げたのだ。")
IPython.display.display(Audio(wav, rate=sr))

### Tacotron + HiFi-GAN (24kHz）

In [ ]:
from ttslearn.pretrained import create_tts_engine

pwg_engine = create_tts_engine("tacotron2_hifipwg_jsut24k", device=device)

%time wav, sr = pwg_engine.tts("あらゆる現実を、すべて自分のほうへねじ曲げたのだ。")
IPython.display.display(Audio(wav, rate=sr))

## JVS 

### Multi-speaker Tacotron + Parallel WaveGAN (16kHz）

In [ ]:
pwg_engine = create_tts_engine("multspk_tacotron2_pwg_jvs16k", device=device)
for spk in ["jvs001", "jvs010", "jvs030", "jvs050", "jvs100"]:
    text = "タコスと寿司、あなたはどっちが好きですか？わたしは" + ("寿司" if random.random() > 0.2 else "タコス") + "が好きです。"
    wav, sr = pwg_engine.tts(text, spk_id=pwg_engine.spk2id[spk])
    print(f"Speaker: {spk}")
    print(text)
    IPython.display.display(Audio(wav, rate=sr))

### Multi-speaker Tacotron + Parallel WaveGAN (24kHz）

In [ ]:
pwg_engine = create_tts_engine("multspk_tacotron2_pwg_jvs24k", device=device)
for spk in ["jvs001", "jvs010", "jvs030", "jvs050", "jvs100"]:
    text = "タコスと寿司、あなたはどっちが好きですか？わたしは" + ("寿司" if random.random() > 0.2 else "タコス") + "が好きです。"
    wav, sr = pwg_engine.tts(text, spk_id=pwg_engine.spk2id[spk])
    print(f"Speaker: {spk}")
    print(text)
    IPython.display.display(Audio(wav, rate=sr))

### Multi-speaker Tacotron + HiFi-GAN (24kHz）

In [ ]:
pwg_engine = create_tts_engine("multspk_tacotron2_hifipwg_jvs24k", device=device)
for spk in ["jvs001", "jvs010", "jvs030", "jvs050", "jvs100"]:
    text = "タコスと寿司、あなたはどっちが好きですか？わたしは" + ("寿司" if random.random() > 0.2 else "タコス") + "が好きです。"
    wav, sr = pwg_engine.tts(text, spk_id=pwg_engine.spk2id[spk])
    print(f"Speaker: {spk}")
    print(text)
    IPython.display.display(Audio(wav, rate=sr))

## Common voice (ja)

### Multi-speaker Tacotron + Parallel WaveGAN (16kHz）

In [ ]:
pwg_engine = create_tts_engine("multspk_tacotron2_pwg_cv16k", device=device)
# NOTE: some speaker's voice have significant amount of noise (e.g., speaker 0)
for spk_id in [5, 6, 12, 15, 19]:
    text = ("今日" if random.random() > 0.5 else "明日") + "の天気は、" +  ("晴れ時々曇り" if random.random() > 0.5 else "晴れ") + "です。"
    wav, sr = pwg_engine.tts(text, spk_id=spk_id)
    print(f"Speaker ID: {spk_id}")
    print(text)
    IPython.display.display(Audio(wav, rate=sr))

### Multi-speaker Tacotron + Parallel WaveGAN (24kHz）

In [ ]:
pwg_engine = create_tts_engine("multspk_tacotron2_pwg_cv24k", device=device)
# NOTE: some speaker's voice have significant amount of noise (e.g., speaker 0)
for spk_id in [5, 6, 12, 15, 19]:
    text = ("今日" if random.random() > 0.5 else "明日") + "の天気は、" +  ("晴れ時々曇り" if random.random() > 0.5 else "晴れ") + "です。"
    wav, sr = pwg_engine.tts(text, spk_id=spk_id)
    print(f"Speaker ID: {spk_id}")
    print(text)
    IPython.display.display(Audio(wav, rate=sr))

## 参考

- Parallel WaveGAN: https://arxiv.org/abs/1910.11480
- HiFi-GAN: https://arxiv.org/abs/2010.05646
- Parallel WaveGANを含むGANベースの非自己回帰型ニューラルボコーダの実装: https://github.com/kan-bayashi/ParallelWaveGAN